# Ephemeris conversion guide

In [1]:
from importlib.metadata import version
import os

try:
    import xradio

    print("XRADIO version", version("xradio"), "already installed.")
except ImportError as e:
    print(e)
    print("Installing XRADIO")

    os.system("pip install xradio")

    import xradio

    print("xradio version", version("xradio"), " installed.")


# These are the three API functions that are currently supported
from xradio.image import load_image, read_image, write_image

XRADIO version 0.0.47 already installed.


## Download dataset

https://open-bitbucket.nrao.edu/projects/CASA/repos/casatestdata/browse/unittest/tclean/venus_ephem_test.ms

In [2]:
import toolviper
ms_file = "venus_ephem_test.ms"
toolviper.utils.data.download(file=ms_file)

[2025-01-16 08:56:37,486]  WARNING    viperlog:  File exists: /Users/jsteeb/miniforge3/envs/zinc/lib/python3.12/site-packages/toolviper/utils/data/.dropbox 
[2025-01-16 08:56:37,486]     INFO    viperlog:  Updating file metadata information ...  
 

 Download List        
 ───────────────────── 
  venus_ephem_test.ms

[2025-01-16 08:56:38,296]     INFO    viperlog:  File exists: venus_ephem_test.ms 


## Convert to Processing Set

In [3]:
from xradio.measurement_set import convert_msv2_to_processing_set


main_chunksize = {"frequency": 1, "time": 20}  # baseline, polarization
outfile = "venus_ephem_test.vis.zarr"
convert_msv2_to_processing_set(
    in_file=ms_file,
    out_file=outfile,
    parallel=False,
    overwrite=True,
    main_chunksize=main_chunksize,
)

[2025-01-16 08:56:38,683]     INFO    viperlog:  Partition scheme that will be used: ['DATA_DESC_ID', 'OBS_MODE', 'OBSERVATION_ID', 'FIELD_ID'] 
[2025-01-16 08:56:38,693]     INFO    viperlog:  Number of partitions: 7 
[2025-01-16 08:56:38,693]     INFO    viperlog:  OBSERVATION_ID [0], DDI [0], STATE [15 22 29 20], FIELD [1], SCAN [ 7 11] 


/Users/jsteeb/miniforge3/envs/zinc/lib/python3.12/site-packages/zarr/core/group.py:2821: UserWarning: Object at WEIGHT is not recognized as a component of a Zarr hierarchy.
  warnings.warn(
/Users/jsteeb/miniforge3/envs/zinc/lib/python3.12/site-packages/zarr/core/group.py:2821: UserWarning: Object at UVW is not recognized as a component of a Zarr hierarchy.
  warnings.warn(
/Users/jsteeb/miniforge3/envs/zinc/lib/python3.12/site-packages/zarr/core/group.py:2821: UserWarning: Object at FLAG is not recognized as a component of a Zarr hierarchy.
  warnings.warn(
/Users/jsteeb/miniforge3/envs/zinc/lib/python3.12/site-packages/zarr/core/group.py:2821: UserWarning: Object at frequency is not recognized as a component of a Zarr hierarchy.
  warnings.warn(
/Users/jsteeb/miniforge3/envs/zinc/lib/python3.12/site-packages/zarr/core/group.py:2821: UserWarning: Object at VISIBILITY is not recognized as a component of a Zarr hierarchy.
  warnings.warn(
/Users/jsteeb/miniforge3/envs/zinc/lib/python3.1

TypeError: Expected a BytesBytesCodec. Got <class 'numcodecs.zstd.Zstd'> instead.

## Processing Set

In [ ]:
ps_name = outfile

from xradio.measurement_set import open_processing_set

ps = open_processing_set(ps_store=outfile)
ps.summary()

In [ ]:
ps.keys()

In [ ]:
ps["venus_ephem_test_0"]

In [ ]:
main_xds = ps["venus_ephem_test_0"]
main_xds.observation_info

In [ ]:
ps["venus_ephem_test_0"].attrs[
    "antenna_xds"
]

In [ ]:
ps["venus_ephem_test_0"].VISIBILITY.attrs[
    "field_and_source_xds"
]

In [ ]:
ps["venus_ephem_test_5"].VISIBILITY.attrs[
    "field_and_source_xds"
].FIELD_PHASE_CENTER.values

In [ ]:
ps.plot_antenna_positions()

In [ ]:
ps.plot_phase_centers()